In [1]:
import sys
import os

os.environ["APPLIED_AI_CONF"] = "../../conf/app.toml"

if os.path.abspath("../..") not in sys.path:
    sys.path.insert(0, os.path.abspath("../.."))

from google.cloud.ml.applied.attributes import attributes
from google.cloud.ml.applied.categories import category
from google.cloud.ml.applied.model import domain_model as m
from google.cloud.ml.applied.images import image_to_text
from google.cloud.ml.applied.marketing import marketing

## Generate Product Categories

Based on any product description and (optional) image, we perform the following:

* Create multimodal embeddings (text and image) from the input description and image
* Perform nearest neighbor search on existing product catalog embeddings in Vertex Vector Search to fetch x nearest product ids
* Lookup existing product categories from the retrieved ids.
* Use an LLM to rank returned categories

In [2]:
category.retrieve_and_rank(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)"
)

CategoryList(values=[['Watches', 'Wrist Watches', 'Timewel Wrist Watches'], ['Watches', 'Wrist Watches', 'Sonata Wrist Watches'], ['Watches', 'Wrist Watches', 'Maserati Time Wrist Watches'], ['Jewellery', 'Necklaces & Chains', 'Necklaces']])

In [3]:
category.retrieve_and_rank(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
    image = "gs://kalschi_etl_2/images/dbdac18a8ee5a8a48238b9685c96e90a.jpg"
)

CategoryList(values=[['Watches', 'Wrist Watches', 'Timewel Wrist Watches'], ['Watches', 'Wrist Watches', 'Sonata Wrist Watches'], ['Watches', 'Wrist Watches', 'Nexus Wrist Watches'], ['Watches', 'Wrist Watches', 'Perucci Wrist Watches'], ['Watches', 'Wrist Watches', 'Maserati Time Wrist Watches'], ['Jewellery', 'Necklaces & Chains', 'Necklaces']])

## Generate Product Attributes

We utilize an LLM to generate product attributes to further enhance a product listing making it more effective for search and recommendations usecases.

Using the same mechanism above, we can use a product description and an image to generate these attributes

In [4]:
attributes.retrieve_and_generate_attributes(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
)

ProductAttributes(product_attributes=[AttributeValue(attribute_name='Mechanism', attribute_value='Quartz'), AttributeValue(attribute_name='Type', attribute_value='Analog'), AttributeValue(attribute_name='Style Code', attribute_value='1100-N1949_S'), AttributeValue(attribute_name='Ideal For', attribute_value='Women'), AttributeValue(attribute_name='Occasion', attribute_value='Casual, Formal, Party-Wedding'), AttributeValue(attribute_name='Power Source', attribute_value='Battery Powered'), AttributeValue(attribute_name='Strap Type', attribute_value='Leather'), AttributeValue(attribute_name='Dial Shape', attribute_value='Rectangle'), AttributeValue(attribute_name='Strap Color', attribute_value='Black'), AttributeValue(attribute_name='Case / Bezel Material', attribute_value='Steel Case'), AttributeValue(attribute_name='Dial Color', attribute_value='Black'), AttributeValue(attribute_name='Strap Material', attribute_value='Genuine Leather Strap'), AttributeValue(attribute_name='Scratch Resis

In [5]:
product_attributes = attributes.retrieve_and_generate_attributes(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
    image = "gs://kalschi_etl_2/images/dbdac18a8ee5a8a48238b9685c96e90a.jpg"
)
product_attributes

ProductAttributes(product_attributes=[AttributeValue(attribute_name='Mechanism', attribute_value='Quartz'), AttributeValue(attribute_name='Type', attribute_value='Analog'), AttributeValue(attribute_name='Style Code', attribute_value='1100-N1949_S'), AttributeValue(attribute_name='Ideal For', attribute_value='Women'), AttributeValue(attribute_name='Occasion', attribute_value='Casual, Formal, Party-Wedding'), AttributeValue(attribute_name='Power Source', attribute_value='Battery Powered'), AttributeValue(attribute_name='Strap Type', attribute_value='Leather'), AttributeValue(attribute_name='Dial Shape', attribute_value='Rectangle'), AttributeValue(attribute_name='Strap Color', attribute_value='Black'), AttributeValue(attribute_name='Case / Bezel Material', attribute_value='Steel Case'), AttributeValue(attribute_name='Dial Color', attribute_value='Black'), AttributeValue(attribute_name='Strap Material', attribute_value='Genuine Leather Strap'), AttributeValue(attribute_name='Scratch Resis

## Generate Product Image Attributes
We can also use an LVM to analyze a product image an extract product attributes

In [6]:
test_image = m.ImageRequest(image = "gs://kalschi_etl_2/images/dbdac18a8ee5a8a48238b9685c96e90a.jpg")
product_attributes = image_to_text.image_to_attributes(test_image)
product_attributes

{'product_attributes': {'brand': 'Timewel', 'style': 'Casual', 'dial_color': 'Silver', 'band_color': 'Black', 'band_material': 'Leather', 'case_shape': 'Round', 'display_type': 'Analog', 'movement': 'Quartz', 'water_resistance': '30m', 'gender': 'Women'}}


ProductAttributes(product_attributes=[AttributeValue(attribute_name='brand', attribute_value='Timewel'), AttributeValue(attribute_name='style', attribute_value='Casual'), AttributeValue(attribute_name='dial_color', attribute_value='Silver'), AttributeValue(attribute_name='band_color', attribute_value='Black'), AttributeValue(attribute_name='band_material', attribute_value='Leather'), AttributeValue(attribute_name='case_shape', attribute_value='Round'), AttributeValue(attribute_name='display_type', attribute_value='Analog'), AttributeValue(attribute_name='movement', attribute_value='Quartz'), AttributeValue(attribute_name='water_resistance', attribute_value='30m'), AttributeValue(attribute_name='gender', attribute_value='Women')])

## Generate Marketing Description
We use the product description as well as the above generated product attributes to generate a product marketing description

In [7]:
marketing_request = m.MarketingRequest(
    description="Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
    attributes=product_attributes.product_attributes
)

marketing_copy = marketing.generate_marketing_copy(marketing_request)
print(marketing_copy.text)

 **Timewel 1100-N1949_S: A Stylish and Functional Analog Watch for Women**

Crafted with precision and elegance, the Timewel 1100-N1949_S analog watch is a must-have accessory for modern women. This watch combines classic design with contemporary features, making it the perfect choice for any occasion.

**Key Features:**

* **Brand:** Timewel, renowned for its commitment to quality and craftsmanship.
* **Style:** Casual and versatile, suitable for everyday wear, office attire, or special events.
* **Dial Color:** Stunning silver dial that adds a touch of sophistication to your look.
* **Band Color:** Sleek black band that complements the silver dial and enhances its overall appeal.
* **Band Material:** Genuine leather band ensures comfort and durability, providing a luxurious feel on your wrist.
* **Case Shape:** Classic round case design that never goes out of style.
* **Display Type:** Analog display with clear markings and easy-to-read hands for precise timekeeping.
* **Movement:** 